# Likely renters vs. likely homeowners

This notebook uses a simple heuristic to determine whether or not a given property has renters or homeowners - just read the M308Assess table and try to match OWN_ADDR and SITE_ADDR.

Since these are freeform text we can't rely on exact matches (for example one entry has 92 AMHERST AVE vs. 92 AMHERST AVENUE) so we need to allow for a fuzzy match.

In [1]:
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import DistanceMetric

from data.connect_db import get_db

con = get_db()

In [2]:
query = "select * from \"M308Assess_CY22_FY23\""
df = pd.read_sql(query, con)

In [3]:
df.head()

,id,PROP_ID,LOC_ID,BLDG_VAL,LAND_VAL,OTHER_VAL,TOTAL_VAL,FY,LOT_SIZE,LS_DATE,...,YEAR_BUILT,BLD_AREA,UNITS,RES_AREA,STYLE,STORIES,NUM_ROOMS,LOT_UNITS,CAMA_ID,TOWN_ID
0,1,R063 002 0017,F_734012_2963106,257400,392000,7300,656700,2023,0.11,20080930,...,1930,1352,1,1352,COLONIAL,2,6,A,28,308
1,2,R015 004 0001,F_730830_2973583,236500,388900,0,625400,2023,0.22,20080422,...,1962,1931,1,1931,RANCH,1,5,A,46,308
2,3,R015 003 0017,F_730659_2973534,171600,382200,5200,559000,2023,0.21,19770301,...,1952,1756,1,1756,CONVENTIONAL,2,7,A,55,308
3,4,R015 005 0001,F_730718_2973342,400200,391900,6100,798200,2023,0.23,20061205,...,1952,2922,1,2922,COLONIAL,2,6,A,64,308
4,5,R015 006 0006,F_730548_2973294,369300,382600,2700,754600,2023,0.21,20050315,...,1952,1792,1,1792,COLONIAL,2,8,A,73,308


## The process

We'll compare the site address and the owner address. If they're close, we'll assume the owner lives onsite.

The comparison itself is just a tokenization of addresses into ngrams of sizes 1 to 3 chars, and the features themselves are counts of each token present in each address.
Then it's just the euclidean distance between the site and owner addresses and a guess of what value should be set as the threshold for when they likely are too dissimilar.

The ngram clusters don't perfectly encode positioning so there's an outside chance we have two streets with the same groups of 1 to 3 letters in different orders, but it is probably not very common.

In [4]:
# get strings
addrs = df["SITE_ADDR"].values + df["OWN_ADDR"].values

# vectorize each
vectorizer = CountVectorizer(analyzer="char", ngram_range=(1, 3))

vectorizer.fit(addrs)

CountVectorizer(analyzer='char', ngram_range=(1, 3))

In [5]:
dist = DistanceMetric.get_metric("euclidean")

def get_dist(a, b):
    
    # vectorize each
    features = vectorizer.transform([a, b])
    
    # measure the distance
    distances = dist.pairwise(features)
    
    return distances[0, 1]

In [6]:
dist = DistanceMetric.get_metric("euclidean")

In [7]:
df['dist'] = df.apply(func=lambda row: get_dist(row.SITE_ADDR, row.OWN_ADDR), axis='columns')

In [8]:
print(df[['dist', "SITE_ADDR", "OWN_ADDR"]].sort_values(by='dist'))

            dist                   SITE_ADDR  \
16492   0.000000          181 -02 ROBBINS ST   
16493   0.000000          181 -03 ROBBINS ST   
16494   0.000000          181 -04 ROBBINS ST   
16464   0.000000           73 -01 BENNETT ST   
16480   0.000000              57 -02 GALE ST   
...          ...                         ...   
4705   12.124356              60  HICKORY DR   
12522  12.124356  1105 -B09 U06 LEXINGTON ST   
3306   12.288206          28  FARNSWORTH AVE   
815    12.409674           335  BEAR HILL RD   
5834   12.806248           976  LEXINGTON ST   

                                 OWN_ADDR  
16492                  181 -02 ROBBINS ST  
16493                  181 -03 ROBBINS ST  
16494                  181 -04 ROBBINS ST  
16464                   73 -01 BENNETT ST  
16480                      57 -02 GALE ST  
...                                   ...  
4705   TEN POST OFFICE SQUARE, 14TH FLOOR  
12522                     73 BARBARA ROAD  
3306         111 LOCUST STR

## Setting a threshold for similarity

Eyeballing the csv output below tells me that at the owner and site addresses start to look dissimilar at about a distance of 6.4, so we'll set anything lower as 'OWNER_OCCUPIED' as true.

In [9]:
columns = [
    "PROP_ID",
    "LOC_ID",
    "SITE_ADDR",
    "OWN_ADDR",
    "dist",
    "OWNER_OCCUPIED"
]

df['OWNER_OCCUPIED'] = df['dist'].map(lambda x: True if x <= 6.4 else False)

df[columns].sort_values(by='dist').to_csv("likely_homeowners.csv", index=False)